In [4]:
#import qiskit tools
import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, Aer, IBMQ
from qiskit.tools.visualization import circuit_drawer
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise

#import python stuff
import matplotlib.pyplot as plt
import numpy as np
import time

# Set backend device, choose to use a simulator
sim = Aer.get_backend('aer_simulator')

# functions..............................................................................................
def make_chsh_circuit(theta_vec):
    """Return a list of QuantumCircuits for use in a CHSH experiemnt
    (one circuit for each value of theta in theta_vec)
    (theta is the angle between the bases of Alice and Bob)
    
        Args:
            theta_vec (list): list of values of angles between the bases of Alice and Bob
        
        Returns:
            List[QuantumCircuit]: CHSH QuantumCircuits for each value of theta
    """
    chsh_circuits = []
    
    for theta in theta_vec:
        obs_vec = ['00', '01', '10', '11'] # observed_vector? ex: '01' means 1st measurement got 0, second got 1
        for el in obs_vec:
            qc = QuantumCircuit(2,2) # create a quantum circuit with two quantum register(qubits) and two classical register
            qc.h(0) # add a H gate on qubit 0
            qc.cx(0, 1) # Add a CX (CNOT) gate on control qubit 0 and target qubit 1
            qc.ry(theta, 0) # rotate around y-axis by theta
            for a in range(2): # what does this do? why do we need this part?
                if el[a] == '1':
                    qc.h(a) 
            qc.measure(range(2),range(2)) # measure q0 save to bit_0, then measure q1 save to bit_1
            chsh_circuits.append(qc)

    return chsh_circuits

def compute_chsh_witness(counts):
    """Computes expectation values for the CHSH inequality, for each
    angle (theta) between measurement axis.

        Args: counts (list[dict]): dict of counts for each experiment
              (4 per value of theta)

        Returns:
            Tuple(List, List): Tuple of lists with the two CHSH witnesses
    """
    # Order is ZZ,ZX,XZ,XX
    
    CHSH1 = []
    CHSH2 = []
    # Divide counts(list of dictionaries) in sets of 4
    for i in range(0, len(counts), 4):  
        theta_dict = counts[i:i + 4]
        zz = theta_dict[0] # alice uses z basis, bob uses z basis, which is the same as a and b measurement.
        zx = theta_dict[1] # a and b'
        xz = theta_dict[2]
        xx = theta_dict[3]
        print(zz)

        no_shots = sum(xx[y] for y in xx)

        chsh1 = 0
        chsh2 = 0

        # E(a,b)
        for element in zz: # zz is a dict
            parity = (-1)**(int(element[0])+int(element[1])) #
            
            print(element)
            
            chsh1+= parity*zz[element]
            chsh2+= parity*zz[element]

        # E(a,b')
        for element in zx:
            parity = (-1)**(int(element[0])+int(element[1]))
            chsh1+= parity*zx[element]
            chsh2-= parity*zx[element]

        # E(a',b)
        for element in xz:
            parity = (-1)**(int(element[0])+int(element[1]))
            chsh1-= parity*xz[element]
            chsh2+= parity*xz[element]

        # E(a',b')
        for element in xx:
            parity = (-1)**(int(element[0])+int(element[1]))
            chsh1+= parity*xx[element]
            chsh2+= parity*xx[element]

        CHSH1.append(chsh1/no_shots)
        CHSH2.append(chsh2/no_shots)
    
    return CHSH1 #return the S of each circuit
# functions...........................................................................................

#split 0 to 2pi into 15 angles. use these angles to build the chsh circuit
number_of_thetas = 15
theta_vec = np.linspace(0,2*np.pi,number_of_thetas)
my_chsh_circuits = make_chsh_circuit(theta_vec)

# test and see the circuit: the circuits create a bell pair, and measure them in different basis. 
# Bob(q1) in +,-,0,1 state, Alice(q0) in theta respected to Bob
# my_chsh_circuits[5].draw()

# Execute and get counts
num_of_shots = 10000
result_ideal = sim.run(my_chsh_circuits, shots=num_of_shots).result()
counts_ideal = result_ideal.get_counts()
CHSH1_ideal = compute_chsh_witness(counts_ideal) # this is the S value

# print(counts_ideal)  

{'11': 4990, '00': 5010}
11
00
{'01': 244, '11': 4743, '10': 241, '00': 4772}
01
11
10
00
{'01': 939, '11': 4136, '10': 971, '00': 3954}
01
11
10
00
{'01': 2004, '11': 3060, '00': 3017, '10': 1919}
01
11
00
10
{'11': 1916, '01': 3014, '00': 1913, '10': 3157}
11
01
00
10
{'11': 940, '01': 4074, '00': 920, '10': 4066}
11
01
00
10
{'11': 245, '00': 267, '10': 4755, '01': 4733}
11
00
10
01
{'10': 5029, '01': 4971}
10
01
{'11': 213, '00': 236, '10': 4774, '01': 4777}
11
00
10
01
{'11': 924, '01': 4092, '00': 900, '10': 4084}
11
01
00
10
{'11': 1932, '01': 3134, '10': 3055, '00': 1879}
11
01
10
00
{'01': 1909, '11': 3070, '00': 3054, '10': 1967}
01
11
00
10
{'01': 955, '11': 4121, '10': 924, '00': 4000}
01
11
10
00
{'01': 260, '11': 4779, '10': 265, '00': 4696}
01
11
10
00
{'11': 4942, '00': 5058}
11
00


In [6]:
print(counts_ideal)
countsx = []
countsx[0]['00'] = counts_ideal[0]['00'] 

[{'11': 4990, '00': 5010}, {'10': 2527, '00': 2518, '01': 2473, '11': 2482}, {'01': 2479, '00': 2462, '10': 2548, '11': 2511}, {'11': 5021, '00': 4979}, {'01': 244, '11': 4743, '10': 241, '00': 4772}, {'11': 1466, '00': 1414, '10': 3577, '01': 3543}, {'01': 1400, '10': 1438, '00': 3550, '11': 3612}, {'01': 240, '10': 228, '00': 4759, '11': 4773}, {'01': 939, '11': 4136, '10': 971, '00': 3954}, {'11': 557, '00': 568, '10': 4419, '01': 4456}, {'01': 586, '11': 4435, '10': 579, '00': 4400}, {'01': 896, '11': 4003, '10': 915, '00': 4186}, {'01': 2004, '11': 3060, '00': 3017, '10': 1919}, {'11': 57, '00': 61, '10': 4943, '01': 4939}, {'01': 57, '11': 4908, '10': 72, '00': 4963}, {'11': 3055, '01': 1944, '00': 3051, '10': 1950}, {'11': 1916, '01': 3014, '00': 1913, '10': 3157}, {'11': 73, '01': 4990, '00': 60, '10': 4877}, {'01': 66, '11': 4856, '10': 53, '00': 5025}, {'11': 1956, '01': 3031, '00': 1942, '10': 3071}, {'11': 940, '01': 4074, '00': 920, '10': 4066}, {'11': 507, '00': 567, '10'

IndexError: list index out of range